In [1]:
import chromedriver_autoinstaller
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import datetime as dt

In [87]:
def rip_navershopping(x):
    soup = BeautifulSoup(x)
    prod_all = soup.select('div.product_item__MDtDF')
    prod_list = []

    for i in prod_all:
        prod_info = {}

        try:
            prod_info['상품명'] = i.select('a.product_link__TrAac')[0].text
        except:
            continue

        try:
            prod_info['상품가격'] = int((i.select('span.price_num__S2p_v')[0].text).replace(",","")[:-1])
        except:
            prod_info['상품가격'] = None

        try:
            temp = i.select('span.price_delivery__yw_We')[0]
            try:
                temp.find('span').decompose()
            except:
                pass
            if temp.text == '무료':
                prod_info['배송비'] = 0
            else:
                prod_info['배송비'] = int(temp.text.replace(",","")[:-1])
        except:
            prod_info['배송비'] = None

        prod_list.append(prod_info)
    
    return prod_list

In [86]:
def navershop_extract(n):
    url = 'https://search.shopping.naver.com/search/all?query=%EC%BD%9C%EB%9D%BC&cat_id=&frm=NVSHATC'
    driver = webdriver.Chrome()
    driver.get(url)

    full_list = []
    idx = 1
    btn = 'a[data-nclick="N=a:pag.page,i:{0}"]'

    while n >= idx:
        for i in range(0, 10):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.5)

        full_list += rip_navershopping(driver.page_source)
        idx += 1

        button = WebDriverWait(driver, 3).until(lambda x:x.find_element(By.CSS_SELECTOR, btn.format(idx)))
        button.click()
    
    return full_list

In [88]:
coke = navershop_extract(30)

In [92]:
df = pd.DataFrame(coke)

In [93]:
df.to_csv('coke.csv', index = False)